In [1]:
import numpy as np
import pandas as pd
import torch

import torch.nn as nn
from torch.utils.data import DataLoader

from Dataset.emb_auto import AutoDataset
from Model.AE import Auto

import utils

import matplotlib.pyplot as plt

In [2]:
# 데이터 다운 & 전처리

df = pd.read_excel('../데이터/더미데이터.xlsx')

try: del df['Unnamed: 0']
except: pass

In [3]:
df[0] = df[0].apply(utils.to_list)
df[1] = df[1].apply(utils.to_list)

In [4]:
len_df = len(df)
train_len = int(len_df *0.6)
val_len = int(len_df * 0.3)

batch_size = 64

train_dataset = AutoDataset(df[:train_len])
train_loader = DataLoader(train_dataset, batch_size=batch_size)

val_dataset = AutoDataset(df[train_len:train_len+val_len])
val_loader = DataLoader(val_dataset, batch_size=batch_size)

test_dataset = AutoDataset(df[train_len+val_len:])
test_loader = DataLoader(test_dataset, batch_size=batch_size)

c:\Users\hkyoo\OneDrive\바탕 화면\SCI\코드\Dataset\emb_auto.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  self.x = torch.FloatTensor(X)


In [5]:
for x,y in train_loader:
  print("X 크기 : {}".format(x.shape))
  print("Y 크기 : {}".format(y.shape))
  break

X 크기 : torch.Size([12, 7])
Y 크기 : torch.Size([12, 7])


In [6]:
input_size = 7
Model = Auto(input_size)

In [7]:
# 데이터 & 모델에 device 붙임!!!
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

model = Model.to(device)

print('작동하는지 실험')
basic_data = torch.rand((1,input_size))
model(basic_data)


cpu is available
작동하는지 실험


(tensor([[0.1095, 0.0000, 0.0000, 0.0000, 0.0000, 0.0694, 0.0204, 0.0364, 0.0193,
          0.0027, 0.0182, 0.0000, 0.0000, 0.1846, 0.1365, 0.3248, 0.1272, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0105, 0.1987, 0.0000, 0.0000, 0.0000, 0.3260,
          0.0262, 0.0000, 0.2100, 0.0000, 0.0000, 0.0676, 0.1857, 0.0000, 0.0214,
          0.0338, 0.0483, 0.0000, 0.0000, 0.0074, 0.3087, 0.0490, 0.0000, 0.0936,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0862, 0.0098, 0.1497,
          0.0459, 0.0000, 0.0714, 0.1472, 0.0090, 0.0000, 0.1057, 0.0244, 0.0454,
          0.0000, 0.0088, 0.0653, 0.0000, 0.0570, 0.0647, 0.0000, 0.0000, 0.0980,
          0.0000, 0.0281, 0.0000, 0.0000, 0.0308, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0565, 0.0000, 0.0262, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0064,
          0.0021, 0.1715, 0.0000, 0.0000, 0.1324, 0.1727, 0.0424, 0.2357, 0.0000,
          0.0000, 0.0887, 0.0244, 0.0345, 0.2940, 0.1133, 0.0000, 0.0000, 0.0000,
          0.0000

In [8]:
criterion = nn.MSELoss()

lr = 1e-3
num_epochs = 1
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [9]:
## 학습하는 코드를 짜봐!!!

train_n = len(train_loader)
val_n = len(val_loader)

# print(train_loader)
best_val_loss = np.inf

for epoch in range(num_epochs):
    print('epoch : {}'.format(epoch))
    running_loss = 0.0
    num = 0
    print("train")
    model.train()
    for inputs, labels in train_loader:  # dataloader 함수에 def __len__에 return 값만큼 반복함
        inputs = inputs.to(device)
        labels = labels.to(device)
        vec, out = model(inputs)
        loss = criterion(out, labels)

        optimizer.zero_grad() #
        loss.backward() # loss가 최소가 되게하는
        optimizer.step() # 어떤 방법으로 learning rate를 움직일거이냐
        running_loss += loss.item() # 한 배치의 loss 더해주고,


    print(running_loss/train_n)

    with torch.no_grad():
              print("Calculating validation results...")
              model.eval()
              val_loss = 0
              for inputs, labels in val_loader:
                  inputs = inputs.to(device)
                  labels = labels.to(device)

                  vec, outs = model(inputs)

                  loss= criterion(outs, labels)
                  val_loss += loss.item()



              # if val_acc > best_val_acc:
              #     print(f"New best model for val accuracy : {val_acc:4.2%}! saving the best model..")
              #     # torch.save(model.module.state_dict(), f"{save_dir}/best.pth")
              #     best_val_acc = val_acc
              # torch.save(model.module.state_dict(), f"{save_dir}/last.pth")
              print("val")
              print(
                  f"loss: {val_loss/val_n:4.8} || "
              )
    print('======================================================')



epoch : 0
train
670541568.0
Calculating validation results...
val
loss: 7.8448179e+08 || 


### 더미데이터 생성

In [84]:
data = []
model = model.requires_grad_(False)
for i, (x,_) in enumerate(test_dataset):
    data.append(str(model(x)[0].tolist()))
    if i==100:
        break

In [87]:
pd.DataFrame(data).to_excel('../데이터/인베딩더미.xlsx')